# 범주형 데이터의 실수화

##### 라벨 인코딩 : 선형/Regression 모델에는 보통 적용 X, 변환된 코드 값들이 각각 가중치를 가져서
#####               서열척도처럼 인식될 수 있기 때문(SVM, LIDGE, LASSO etc.), 트리 계열 모델에는 상관 없다.
#####               일반적으로 get_dummies를 많이 쓰다고 생각하면 됨.
##### 
##### TF-IDF, Word to vec 등 문서 단어 행렬변환방식의 전처리는 추천시스템, 텍스트 분석 등에서 활용됨

In [1]:
# import libraries and create dataset
import pandas as pd           # importing pandas package
from sklearn.preprocessing import LabelEncoder      # 범주형 데이터의 실수화 함수
from sklearn.preprocessing import OneHotEncoder     # 더미변수 생성, 가변환 함수 임포트


# 데이터 set 생성 및 확인
x_train = pd.DataFrame(["남성", "여성", "남성", "여성", "남성", "여성"], columns=["성별"])

In [2]:
#데이터 확인
x_train.head(3)

,성별
0,남성
1,여성
2,남성


In [3]:
# 데이터 프레임의 정보 확인 : object 타입 '성별' 변수
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   성별      6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


In [4]:
# 범주형 데이터의 갯수 확인
x_train['성별'].value_counts()

남성    3
여성    3
Name: 성별, dtype: int64

#### 레이블 인코딩 : 문자를 0부터 시작하는 정수형 숫자로 변환

In [5]:
# 레이블 인코더 생성
encoing = LabelEncoder()

# x_train 데이터를 이용하여 피팅하고 레이블 숫자로 변환
encoing.fit(x_train["성별"])
x_train["성별_인코딩"] = encoing.transform(x_train["성별"])

In [6]:
#인코딩 결과 확인
print(x_train)

# 클래스 확인
print(list(encoing.classes_))
# 인코딩 값으로 문자값 확인
print(list(encoing.inverse_transform([1,0])))

   성별  성별_인코딩
0  남성       0
1  여성       1
2  남성       0
3  여성       1
4  남성       0
5  여성       1
['남성', '여성']
['여성', '남성']


# One-Hot Encoding

#### 더미변수 생성, 가변환

In [7]:
# 데이터 확인(실수화한 데이터 사용)
x_train.head(2)

,성별,성별_인코딩
0,남성,0
1,여성,1


In [8]:
# 원핫인코더 생성
# Sparse를 True로 할 경우 "(행, 열) 1"의 좌표리스트의 형식, False로 할 경우 넘파이 배열로 반환
one_encoding = OneHotEncoder(sparse=False)

# x_train 데이터를 이용하여 피팅
one_encoding.fit(x_train[["성별"]])

# 가변환값 변환
one_encoding.transform(x_train[["성별"]])

# one_encoding.fit_transform(X_train[["성별"]])

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [9]:
# 가변환된 피쳐 확인
print(one_encoding.get_feature_names_out())

['성별_남성' '성별_여성']


### 인코딩 결과 확인

In [10]:
# 가변환된 값을 x_train_one 데이터 프레임으로 저장
x_train_one = pd.DataFrame(one_encoding.transform(x_train[['성별']]),
                           columns=['성별0', '성별1'])

# x_train 데이터와 x_train_one의 가변환된 속성 합침
x_train = pd.concat([x_train, x_train_one], axis=1)

# 결과 확인
x_train.head(3)

,성별,성별_인코딩,성별0,성별1
0,남성,0,1.0,0.0
1,여성,1,0.0,1.0
2,남성,0,1.0,0.0


# 텍스트 데이터 실수화 

### 단어의 빈도수를 count하여 단어를 수치화하는 방법

In [11]:
# CountVectorizer: 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 Bag of Words로 인코딩하는 함수

# import libraries
from sklearn.feature_extraction.text import CountVectorizer

# Create dataset
corpus = [
    '청년 인재 개발 양성 과정', '인공지는 청년 양성', '미래 인공지능 데이터 대한민국',
    '데이터 원유 기술사 청년 개발'
]

print(corpus)
print(type(corpus))

['청년 인재 개발 양성 과정', '인공지는 청년 양성', '미래 인공지능 데이터 대한민국', '데이터 원유 기술사 청년 개발']
<class 'list'>


In [12]:
# 텍스트 실수화 : 단어의 빈도수 count

# 카운트 백터라이저 생성
count_vect = CountVectorizer()

# 단어 카운트
count_vect.fit(corpus)

# 단어 확인
count_vect.vocabulary_

{'청년': 11,
 '인재': 10,
 '개발': 0,
 '양성': 6,
 '과정': 1,
 '인공지는': 8,
 '미래': 5,
 '인공지능': 9,
 '데이터': 4,
 '대한민국': 3,
 '원유': 7,
 '기술사': 2}

In [13]:
# 단어 백터화한 값을 array로 변환하여 확인
count_vect.transform(corpus).toarray()

array([[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1]], dtype=int64)

In [14]:
# 단어 백터화
features = count_vect.transform(corpus)
features

<4x12 sparse matrix of type '<class 'numpy.int64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [15]:
# 문서단어행렬로 변환

# 속성 이름만 변환
vocab = count_vect.get_feature_names_out()
vocab

array(['개발', '과정', '기술사', '대한민국', '데이터', '미래', '양성', '원유', '인공지는', '인공지능',
       '인재', '청년'], dtype=object)

In [16]:
# 문서단어행렬(DTM)을 데이터 프레임으로 변환
DTM = pd.DataFrame(features.toarray(), columns = vocab).head()
DTM

,개발,과정,기술사,대한민국,데이터,미래,양성,원유,인공지는,인공지능,인재,청년
0,1,1,0,0,0,0,1,0,0,0,1,1
1,0,0,0,0,0,0,1,0,1,0,0,1
2,0,0,0,1,1,1,0,0,0,1,0,0
3,1,0,1,0,1,0,0,1,0,0,0,1


In [17]:
# 문서단어행렬에 일치하는 단어 확인
count_vect.transform(['기술사 대한민국 인재 만세']).toarray()

array([[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0]], dtype=int64)

### 단어의 빈도수를 단어들마다 중요한 정도를 가중치로 주어 수치화하는 방법

In [18]:
# TfidfVectorizer : TF-IDF 방식으로 단어의 가중치를 조정한 BOW로 인코딩하는 함수

# import libraries
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
# create and check dataset
# 단어의 빈도수 수치화에서 활용한 corpus 데이터셋 사용

corpus

['청년 인재 개발 양성 과정', '인공지는 청년 양성', '미래 인공지능 데이터 대한민국', '데이터 원유 기술사 청년 개발']

In [20]:
# 텍스트 실수화 : TF-IDF 방식

# TF-IDF 벡터라이저 생성
tfid = TfidfVectorizer()

# 단어 카운트
tfid.fit(corpus)

# 단어사전 확인
tfid.vocabulary_

{'청년': 11,
 '인재': 10,
 '개발': 0,
 '양성': 6,
 '과정': 1,
 '인공지는': 8,
 '미래': 5,
 '인공지능': 9,
 '데이터': 4,
 '대한민국': 3,
 '원유': 7,
 '기술사': 2}

In [21]:
# 단어 벡터화한 값을 array로 변환하여 확인
tfid.transform(corpus).toarray()

array([[0.41263976, 0.52338122, 0.        , 0.        , 0.        ,
        0.        , 0.41263976, 0.        , 0.        , 0.        ,
        0.52338122, 0.33406745],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.55349232, 0.        , 0.70203482, 0.        ,
        0.        , 0.44809973],
       [0.        , 0.        , 0.        , 0.52547275, 0.41428875,
        0.52547275, 0.        , 0.        , 0.        , 0.52547275,
        0.        , 0.        ],
       [0.41263976, 0.        , 0.52338122, 0.        , 0.41263976,
        0.        , 0.        , 0.52338122, 0.        , 0.        ,
        0.        , 0.33406745]])

In [22]:
# 단어 벡터화
features_idf = tfid.transform(corpus)
features_idf

<4x12 sparse matrix of type '<class 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [23]:
# 문서단어행렬로 변환

# 속성 이름만 반환
vocab_idf = count_vect.get_feature_names_out()
vocab_idf

array(['개발', '과정', '기술사', '대한민국', '데이터', '미래', '양성', '원유', '인공지는', '인공지능',
       '인재', '청년'], dtype=object)

In [24]:
# 문서단어행렬(DTM)을 데이터 프레임으로 변환
DTM_idf = pd.DataFrame(features_idf.toarray(), columns = vocab_idf).head()
DTM_idf

,개발,과정,기술사,대한민국,데이터,미래,양성,원유,인공지는,인공지능,인재,청년
0,0.41264,0.523381,0.000000,0.000000,0.000000,0.000000,0.412640,0.000000,0.000000,0.000000,0.523381,0.334067
1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.553492,0.000000,0.702035,0.000000,0.000000,0.448100
2,0.00000,0.000000,0.000000,0.525473,0.414289,0.525473,0.000000,0.000000,0.000000,0.525473,0.000000,0.000000
3,0.41264,0.000000,0.523381,0.000000,0.412640,0.000000,0.000000,0.523381,0.000000,0.000000,0.000000,0.334067
